In [1]:
%load_ext autoreload
%autoreload 2


In [1]:
import sys
sys.path.append("..")
from spark.spark_config import create_spark_session
spark = create_spark_session()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-78aa02d8-346b-44bd-8bb5-bbcd9120ad83;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	fou

In [5]:
spark

In [3]:
from src.data_loader import DataLoader
from src.preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from src.feature_extractor import FeatureExtractor


In [4]:
data_loader = DataLoader(spark)
df = data_loader.load_sample()

In [23]:
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])


In [6]:
df = Preprocessor.run(df)


In [7]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [9]:
extractor = FeatureExtractor(vocabSize=10000, minDF=5.0)
# extractor.fit(df_tokenized)
# df_final = extractor.transform(df_tokenized)
# df_tokenized.unpersist()

In [15]:
from pathlib import Path
base_dir = Path.cwd().parent
test_file_path = base_dir / "models" / "test" / "tf-idf"
# extractor.save(str(test_file_path))
extractor.load(str(test_file_path))

In [16]:
df_final.show(3)

AttributeError: 'FeatureExtractor' object has no attribute 'show'